<a href="https://colab.research.google.com/github/dudeh534/Mirae_Exit/blob/main/%EA%B8%B0%EC%88%A0%EC%A0%81%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기술적 분석 백테스팅

1. 분석 대상 데이터
2012~2022년까지의 코스피200 종목 일봉데이터

2. 백테스팅 규칙
  매수조건: 골든크로스(5,20), 골든크로스(10,20) 이면서 거래량이 전일자의 3배일 때
  매도조건: 데드크로스(5,20) 일 때

참고 사이트:
https://aplab.tistory.com/entry%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%A3%BC%EC%8B%9D-%EB%B0%B1%ED%85%8C%EC%8A%A4%ED%8A%B8


# 1. 개발에 필요한 필수 라이브러리 설치

In [1]:
!pip install backtrader
!pip install pykrx
!pip install finance-datareader
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 17.2 MB/s eta 0:00:00


# 2. 코스피200종목의 2012~2022년 일봉 데이터 수집

In [2]:
import pandas as pd
from pykrx import stock
import FinanceDataReader as fdr

# KOSPI 200 지수에 속한 종목 리스트 가져오기
# KOSPI 200 지수 코드: 1028
kospi200_tickers = stock.get_index_portfolio_deposit_file("1028")

# 티커별로 데이터 가져온 후 티커명.csv의 엑셀파일로 저장
for ticker in kospi200_tickers:
    df = fdr.DataReader(ticker, '2012-01-01', '2022-12-31')
    df.to_csv(f'{ticker}.csv')

print("Data saved to CSV files")

Data saved to CSV files


# 3. backtrader 라이브러리 형식에 맞게 엑셀파일의 형식을 변환

In [3]:
import backtrader.feeds as btfeeds

class customCSV(btfeeds.GenericCSVData):
    params=(
        ('dtformat', '%Y-%m-%d'),
        ('datetime', 0),
        ('open', 1),
        ('high', 2),
        ('low', 3),
        ('close', 4),
        ('volume', 5),
        )

# 4. backtrader 라이브러리를 활용해 백테스팅 관련 전략 작성

In [9]:
from datetime import datetime
import backtrader as bt

# 이동평균선 & 거래량 돌파 관련 전략 클래스
class SmaCross(bt.Strategy):

    # 이동평균선(3,5,10,20,60), 거래량 변수에 데이터 할당
    def __init__(self):
        self.sma3 = bt.indicators.SimpleMovingAverage(period=3)
        self.sma5 = bt.indicators.SimpleMovingAverage(period=5)
        self.sma10 = bt.indicators.SimpleMovingAverage(period=10)
        self.sma20 = bt.indicators.SimpleMovingAverage(period=20)
        self.sma60 = bt.indicators.SimpleMovingAverage(period=60)
        self.holding  = 0

    # 매수 매도 전략 작성
    def next(self):

      # 매수조건

        # 잔고가 비었을 때
        if not self.position:
            # 당일거래량이 전일거래량 3배 일 때
          if self.data.volume[0] >= self.data.volume[-1] * 3:
            # 골든크로스(5,10) 이면서 골든크로스(10,20) 일 때
            if self.sma5 > self.sma20 and self.sma10 > self.sma20:
              # 현재 보유한 현금
              cash = self.broker.get_cash()
              # 주문 가능 수량 계산
              size = int(cash / self.data.close)
              if size > 0:
                self.buy(size=size)

      # 매도조건

        #데드크로스(5,20) 일 때 매도
        elif self.sma5 < self.sma20:
          self.close()

    # 주문 실행 후 관련 로그를 출력하기 위한 함수
    def notify_order(self, order):
        if order.status not in [order.Completed]:
            return

        if order.isbuy():
            action = 'Buy'
        elif order.issell():
            action = 'Sell'

        stock_price = self.data.close[0]
        cash = self.broker.getcash()
        value = self.broker.getvalue()
        self.holding += order.size

        # 매수/도, 잔고수량, 주문가격, 예수금, 평가금액 출력
        print('%s[%d] holding[%d] price[%f] cash[%.2f] value[%.2f]'
              % (action, abs(order.size), self.holding, stock_price, cash, value))


# 5. 백테스팅을 실행하는 메인 함수

In [10]:
# 종목갯수 변수 초기화
num = 0

# 200개 종목에 대해 반복문을 돌며 백테스팅 수행
for ticker in kospi200_tickers:
    합산수익률 = 0
    평균수익률 = 0

    # 백테스팅을 위한 "Cerebro" 객체 생성
    cerebro = bt.Cerebro()

    # 예수금, 수수료 설정
    cerebro.broker.setcash(999999999)
    cerebro.broker.setcommission(0.000)

    print(f"Backtesting for {ticker}")

    # cerebro 객체에 분석할 데이터와 적용할 전략 정의
    data=customCSV(dataname=f'{ticker}.csv')
    cerebro.adddata(data)
    cerebro.addstrategy(SmaCross)  # Add the trading strategy

    # 백테스팅 수행 및 전,후 평가금액 저장
    start_value = cerebro.broker.getvalue()
    cerebro.run()
    final_value = cerebro.broker.getvalue()

    수익률 = (final_value - start_value) / start_value * 100.0
    합산수익률 = 합산수익률 + 수익률
    print("결과")
    print('* start value : %s won' % str(start_value))
    print('* final value : %s won' % str(final_value))
    print('* earning rate : %.2f %%' % ((final_value - start_value) / start_value * 100.0))

    # 백테스팅할 종목 갯수 설정
    num = num + 1
    if num == 20:
        print("Backtesting end!")
        break
    cerebro.plot()

종목갯수 = str(num)
평균수익률 = str(합산수익률/num)

print( "종목갯수:" + 종목갯수 )
print( "평균수익률" + 평균수익률)

Backtesting for 005930
Buy[35038] holding[35038] price[28500.000000] cash[4920799.00] value[1003503799.00]
Sell[35038] holding[0] price[27780.000000] cash[993693159.00] value[993693159.00]
Buy[39183] holding[39183] price[24620.000000] cash[10199859.00] value[974885319.00]
Sell[39183] holding[0] price[25280.000000] cash[1007799039.00] value[1007799039.00]
Buy[19015] holding[19015] price[53000.000000] cash[1007799039.00] value[2015594039.00]
Sell[19015] holding[0] price[49200.000000] cash[1962352039.00] value[1962352039.00]
결과
* start value : 999999999 won
* final value : 1962352039.0 won
* earning rate : 96.24 %


<IPython.core.display.Javascript object>

Backtesting for 373220
결과
* start value : 999999999 won
* final value : 999999999.0 won
* earning rate : 0.00 %
Backtesting for 000660
Buy[37735] holding[37735] price[26600.000000] cash[5682749.00] value[1009433749.00]
Sell[37735] holding[0] price[27800.000000] cash[1058489249.00] value[1058489249.00]
Buy[37601] holding[37601] price[27100.000000] cash[1901149.00] value[1020888249.00]
Sell[37601] holding[0] price[28950.000000] cash[1075409699.00] value[1075409699.00]
Buy[31958] holding[31958] price[31950.000000] cash[14404099.00] value[1035462199.00]
Sell[31958] holding[0] price[32100.000000] cash[1049843299.00] value[1049843299.00]
Buy[39393] holding[39393] price[25750.000000] cash[19716349.00] value[1034086099.00]
Sell[39393] holding[0] price[26400.000000] cash[1034086099.00] value[1034086099.00]
Buy[12443] holding[12443] price[82600.000000] cash[2561399.00] value[1030353199.00]
Sell[12443] holding[0] price[77400.000000] cash[963160999.00] value[963160999.00]
Buy[10701] holding[10701]